In [1]:
import Pkg
Pkg.update()

pkgs = [
    "Revise",
    "MetaGraphs",
    "Graphs",
    "JSON",
    "uCSV",
    "DataFrames",
    "Dates",
    "Primes",
    "Kmers",
    "BioSequences",
    "FASTX"
]

Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

# Pkg.add(url="https://github.com/cjprybol/Mycelia.git", rev="master")
# Pkg.develop(path="$(homedir())/workspace/Mycelia")
import Mycelia

    Updating registry at `/opt/julia/registries/General`
  No Changes to `~/work/Mycelia/Project.toml`
  No Changes to `~/work/Mycelia/Manifest.toml`
Precompiling project...
  ✓ Mycelia
  1 dependency successfully precompiled in 20 seconds (232 already precompiled)
   Resolving package versions...
  No Changes to `~/work/Mycelia/Project.toml`
  No Changes to `~/work/Mycelia/Manifest.toml`


In [2]:
#papermill parameters
base_directory = "$(homedir())/workspace/sars-cov2-pangenome-analysis/"
sequences_directory = "$(base_directory)/data/sequences"
metadata_file = "$(base_directory)/metadata/sequences.csv"
working_directory = "$(homedir())/workspace/scratch/$(Dates.today())"

"/home/jovyan/workspace/scratch/2022-09-11"

In [3]:
mkpath(working_directory)
cd(working_directory)

In [4]:
# f = metadata_file
f = open(`head $metadata_file`)
@time sequence_metadata = DataFrames.DataFrame(uCSV.read(f, quotes='"', header=1, typedetectrows=100)...)

  3.718736 seconds (7.92 M allocations: 481.020 MiB, 3.33% gc time, 76.04% compilation time)


,Accession,SRA_Accession,Submitters
,String,String,String
1,NC_045512.2,,"Wu,F., Zhao,S., Yu,B., Chen,Y.M., Wang,W., Song,Z.G., Hu,Y., Tao,Z.W., Tian,J.H., Pei,Y.Y., Yuan,M.L., Zhang,Y.L., Dai,F.H., Liu,Y., Wang,Q.M., Zheng,J.J., Xu,L., Holmes,E.C., Zhang,Y.Z., Baranov,P.V., Henderson,C.M., Anderson,C.B., Gesteland,R.F., Atkins,J.F., Howard,M.T., Robertson,M.P., Igel,H., Baertsch,R., Haussler,D., Ares,M. Jr., Scott,W.G., Williams,G.D., Chang,R.Y., Brian,D.A., Chen,Y.-M., Song,Z.-G., Tao,Z.-W., Tian,J.-H., Pei,Y.-Y., Zhang,Y.-L., Dai,F.-H., Wang,Q.-M., Zheng,J.-J., Zhang,Y.-Z."
2,OP164776.1,,"Afrad Moon,M.H., Qadri,F."
3,OP164777.1,,"Afrad Moon,M.H., Qadri,F."
4,OP164778.1,,"Afrad Moon,M.H., Qadri,F."
5,OP164779.1,,"Afrad Moon,M.H., Qadri,F."
6,OP164780.1,,"Afrad Moon,M.H., Qadri,F."
7,OP164781.1,,"Afrad Moon,M.H., Qadri,F."
8,OP164782.1,,"Afrad Moon,M.H., Qadri,F."
9,OP164783.1,,"Afrad Moon,M.H., Qadri,F."


In [5]:
fastx_files = map(x -> "$(sequences_directory)/$(x).fna", sequence_metadata[!, "Accession"])

9-element Vector{String}:
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/NC_045512.2.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164776.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164777.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164778.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164779.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164780.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164781.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164782.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164783.1.fna"

In [39]:
# kmer_size = Mycelia.assess_dnamer_saturation(fastx_files)
# kmer_size = 11
kmer_size = 17

17

In [40]:
# import BioSequences
# import FASTX
# import Kmers

In [41]:
graph = Mycelia.initialize_graph()
graph = Mycelia.add_fastx_to_graph!(graph, fastx_files)

{9, 0} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [42]:
# add fastx metadata to graphs

# for each fastx in the graph, add the metadata
Mycelia.add_metadata_from_table!(graph, sequence_metadata, identifier_column = "Accession")

{9, 0} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [43]:
# for kmer_size in kmer_sizes
function add_fasta_record_kmers_to_graph!(graph, kmer_size)
    record_vertices = collect(MetaGraphs.filter_vertices(graph, :TYPE, FASTX.FASTA.Record))
    for vertex in record_vertices
        record_identifier = graph.vprops[vertex][:identifier]
        record_sequence = graph.vprops[vertex][:sequence]
        kmer_counts = Mycelia.count_canonical_kmers(Kmers.DNAKmer{kmer_size}, record_sequence)
        Mycelia.add_kmers_to_graph!(graph, keys(kmer_counts))
        Mycelia.add_record_kmer_counts_to_graph!(graph, kmer_counts, record_identifier)
        Mycelia.add_record_edgemers_to_graph!(graph, record_identifier, kmer_size)
    end
    return graph
end

add_fasta_record_kmers_to_graph! (generic function with 1 method)

In [44]:
# link kmer_counts to record

In [45]:
# {30145, 288695} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)
add_fasta_record_kmers_to_graph!(graph, kmer_size)

{31169, 293711} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [46]:
outfile = joinpath(working_directory, "$(kmer_size).gfa")

"/home/jovyan/workspace/scratch/2022-09-11/17.gfa"

In [47]:
open(outfile, "w") do io
    println(io, "H\tVN:Z:1.0")
    kmer_vertices = collect(MetaGraphs.filter_vertices(graph, :TYPE, Kmers.kmertype(Kmers.DNAKmer{kmer_size})))
    record_vertices = collect(MetaGraphs.filter_vertices(graph, :TYPE, FASTX.FASTA.Record))
    for vertex in kmer_vertices
        if haskey(graph.vprops[vertex], :kmer)
            sequence = graph.vprops[vertex][:kmer]
        else
            sequence = graph.vprops[vertex][:sequence]
        end
        # depth = graph.vprops[vertex][:count]
        # depth = length(graph.vprops[vertex][:evidence])
        total_count = 0
        vertex_outneighbors = Graphs.outneighbors(graph, vertex)
        for connected_record in intersect(vertex_outneighbors, record_vertices)
            edge = Graphs.Edge(vertex, connected_record)
            total_count += MetaGraphs.get_prop(graph, edge, :count)
        end
        
        fields = ["S", "$vertex", sequence, "DP:f:$(total_count)"]
        line = join(fields, '\t')
        println(io, line)
    end
    edgemer_edges = collect(MetaGraphs.filter_edges(graph, :TYPE, Kmers.kmertype(Kmers.DNAKmer{kmer_size+1})))
    overlap = kmer_size - 1
    for edgemer_edge in edgemer_edges
        source_kmer, dest_kmer = Mycelia.edgemer_to_vertex_kmers(MetaGraphs.get_prop(graph, edgemer_edge, :sequence))
        link = ["L",
                    edgemer_edge.src,
                    BioSequences.iscanonical(source_kmer) ? '+' : '-',
                    edgemer_edge.dst,
                    BioSequences.iscanonical(dest_kmer) ? '+' : '-',
                    "$(overlap)M"]
        line = join(link, '\t')
        println(io, line)
    end
end

In [ ]:
# defunct bcalm usage
# run(`conda install -c bioconda bcalm`)

# fasta_file_list = joinpath(working_directory, repr(hash(fastx_files)) * ".fasta_list.txt")
# open(fasta_file_list, "w") do io
#     for f in fastx_files
#         @show f
#         println(io, f)
#     end
# end

# k = 3
# outfile = fasta_file_list * ".bcalm.$(k).fna"
# cmd = `bcalm -in $(fastx_files[1]) -abundance-min 1 -kmer-size 11 -all-abundance-counts -out $(outfile)`
# run(cmd)

# cmds = [
#     `bcalm`,
#     `-in $(fasta_list_file)`,
#     `-abundance-min 1`,
#     `-kmer-size 3`,
#     `-all-abundance-counts`,
#     `-abundance-max $(typemax(UInt64))`
# ]
# run(cmds)

# ls -1 *.fastq > list_reads
# ./bcalm -in list_reads [..]
# ./bcalm -in [reads.fa] -kmer-size [kmer_size] -abundance-min [abundance_threshold]

# scripts/convertToGFA.py

In [ ]:
# for (index, metadata) in graph.vprops
#     @show index
#     for (key, value) in metadata
#         @show key, value
#     end
# end